In [23]:
%cd ~/Desktop/WallStreetQuants_quantskillstraining/quant_project_1
%run notebooks/00_config_and_utils.ipynb

/Users/wangzhenghua/Desktop/WallStreetQuants_quantskillstraining/quant_project_1


In [24]:
tickers = [
    "AAPL","MSFT","AMZN","GOOGL","NVDA","META","TSLA","BRK-B","JPM","JNJ",
    "PG","KO","XOM","V","MA","UNH","HD","DIS","PFE","CSCO",
    "BAC","WMT","CRM","AVGO","COST","ABBV","LLY","TMO","MCD","PEP",
    "NFLX","ADBE","CMCSA","INTC","QCOM","ORCL","TXN","NKE","LIN","ACN",
    "DHR","AMD","UPS","PM","NEE","WFC","LOW","UNP","IBM","CVX"
]
benchmark = "SPY" # use to estimate beta and port regression
len(tickers)

50

In [35]:
import yfinance as yf
# have to put auto_adjust = False
data = yf.download(tickers + [benchmark], start=CONFIG["start"], end=CONFIG["end"], auto_adjust=False)

adj = data["Adj Close"].copy()
ret = adj / adj.shift() - 1

ret = ret.dropna(how="all") # remove dates when ticker is missing

min_obs = int(0.95 * ret.shape[0]) # keep ticker that have at least 95% of available prices
good_cols = ret.columns[ret.notna().sum() >= min_obs] # keep columns with enough history

adj = adj[good_cols]
ret = ret[good_cols]

spy_ret = ret[benchmark].copy()
ret = ret.drop(columns=[benchmark])
adj = adj.drop(columns=[benchmark])

adj.to_parquet("data/processed/adj_close.parquet") # save it to processed
ret.to_parquet("data/processed/ret.parquet") 
spy_ret.to_frame("SPY").to_parquet("data/processed/spy_ret.parquet")

ret.shape, ret.columns[:10]

[*********************100%***********************]  51 of 51 completed


((2512, 50),
 Index(['AAPL', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BAC', 'BRK-B',
        'CMCSA'],
       dtype='object', name='Ticker'))

In [38]:
print("Ret date range:", ret.index.min(), "to", ret.index.max())
print("Num stocks:", ret.shape[1])

missing_frac = ret.isna().mean().sort_values(ascending=False).head(10)
missing_frac
# the fact that being strict and nothing gets dropped shows how missing values are a rarity in large cap stocks

Ret date range: 2016-01-05 00:00:00 to 2025-12-30 00:00:00
Num stocks: 50


Ticker
AAPL    0.0
PG      0.0
MCD     0.0
META    0.0
MSFT    0.0
NEE     0.0
NFLX    0.0
NKE     0.0
NVDA    0.0
ORCL    0.0
dtype: float64